In [1]:
!pip install vinorm
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datasets import load_dataset

ds = load_dataset("tmnam20/Vietnamese-News", "dantri")

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 100396/100396 [00:03<00:00, 31828.85 examples/s]


In [6]:
ds["train"]["content"][:5]

['Ngày 4/12, Sở Lao động - Thương binh và Xã hội tỉnh Quảng Nam có buổi làm việc với trường Cao đẳng Thaco về công tác tuyển sinh, đào tạo, giải quyết việc làm, đưa lao động đi làm việc ở nước ngoài theo hợp đồng trong thời gian qua và những năm đến.\nLãnh đạo Sở Lao động - Thương binh và Xã hội tỉnh Quảng Nam làm việc với lãnh đạo trường Cao đẳng Thaco (Ảnh: Công Bính).\nTại buổi làm việc, ông Phan Tiềm - Hiệu trưởng trường Cao đẳng Thaco - cho hay từ năm 2021 đến nay, nhà trường đã phối hợp tốt với Trung tâm Dịch vụ việc làm trực thuộc Sở Lao động - Thương binh và Xã hội và UBND các huyện của tỉnh Quảng Nam giáp biên giới với Lào để tư vấn, tuyển sinh đào tạo và bố trí việc làm cho hơn 310 lao động là người dân tộc thiểu số.\nTheo đó, người lao động được công ty ký hợp đồng dài

In [24]:
import pandas as pd
df = pd.DataFrame(ds["train"]["content"], columns=["original"])

In [25]:
df_unique = df.copy()
df_unique['original'] = df_unique['original'].str.split('\n')
df_unique = df_unique.explode('original').reset_index(drop=True)

In [26]:
df_unique.describe()

,original
count,1745310
unique,1080741
top,Bưu điện
freq,1327


In [27]:
df_unique = df_unique.drop_duplicates()
df_unique.describe()

,original
count,1080741
unique,1080741
top,"Theo ông Nguyễn Trương Thành, Giám đố..."
freq,1


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100396 entries, 0 to 100395
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   original  100396 non-null  object
dtypes: object(1)
memory usage: 784.5+ KB


In [34]:

df_unique['norm'] = df_unique['original']
df_unique.head()

,original,norm
0,"Ngày 4/12, Sở Lao động - Thương binh va...","Ngày 4/12, Sở Lao động - Thương binh va..."
1,Lãnh đạo Sở Lao động - Thương binh và...,Lãnh đạo Sở Lao động - Thương binh và...
2,"Tại buổi làm việc, ông Phan Tiềm - Hi...","Tại buổi làm việc, ông Phan Tiềm - Hi..."
3,"Theo đó, người lao động được công ty...","Theo đó, người lao động được công ty..."
4,Từ việc làm trên đã góp phần giải q...,Từ việc làm trên đã góp phần giải q...


In [38]:

import re

def preprocess_text(text):
  """
  Adds whitespace between text and numbers, and replaces hyphens with spaces.
  """
  text = re.sub(r'(\d+)([a-zA-Z]+)', r'\1 \2', text)
  text = re.sub(r'([a-zA-Z]+)(\d+)', r'\1 \2', text)
  
  text = text.replace("-", " ")
  
  return text

df_unique['norm'] = df_unique['original'].apply(preprocess_text)
df_unique.head()


,original,norm
0,"Ngày 4/12, Sở Lao động - Thương binh va...","Ngày 4/12, Sở Lao động Thương binh va..."
1,Lãnh đạo Sở Lao động - Thương binh và...,Lãnh đạo Sở Lao động Thương binh và...
2,"Tại buổi làm việc, ông Phan Tiềm - Hi...","Tại buổi làm việc, ông Phan Tiềm Hi..."
3,"Theo đó, người lao động được công ty...","Theo đó, người lao động được công ty..."
4,Từ việc làm trên đã góp phần giải q...,Từ việc làm trên đã góp phần giải q...


In [39]:
dlist = df_unique["norm"].to_list()

In [40]:
dlist[10:30]

['"Bình quân mỗi tháng thu nhập của tôi tính ra tiền Việt khoảng 50 70 triệu đồng, vào mùa cao điểm thu nhập có thể nhiều hơn. Trừ tiền thuê nhà, chi phí sinh hoạt, tôi cũng để dành được 40 60 triệu đồng. Lao động làm ở mỗi tàu khác nhau, thu nhập cũng sẽ khác", anh Tuấn cho biết.',
 'Lao động Việt ghi lại cảnh nhúng tay vào nước sôi cho đỡ lạnh khi đi biển (Clip: Nhân vật cung cấp).',
 'Tuy nhiên, theo anh Tuấn, để có mức thu nhập như vậy, mỗi ngày anh phải lênh đênh trên biển 15 17 tiếng. So với những công việc trên bờ, anh cho biết, nghề đi biển vất vả gấp nhiều lần. Khó khăn lớn nhất là phải đối mặt với thời tiết khắc nghiệt.',
 '"Mùa đông, có những hôm tuyết rơi trắng trời, cơ thể có thể giữ ấm được nhưng đôi bàn tay bắt buộc phải chịu lạnh. Anh em chúng tôi luôn cố gắng giữ ch

In [42]:

from vinorm import TTSnorm
def convert_numbers_to_spoken(text):
  """
  Converts numbers in the text to spoken language using Vinorm.
  """
  numbers = re.findall(r'\d+', text)
  for number in numbers:
    spoken_number = TTSnorm(number)
    text = text.replace(number, spoken_number)
  
  return text


df_unique['norm'] = df_unique['norm'].apply(convert_numbers_to_spoken)
df_unique.head()